In [3]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets
from scipy import constants as C

base_diagnostic = 'out/test'
searay_object_name = 'vacuum'
searay_wave_type = 'uppe'
time_domain = True
real_field = True

mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.size'] = 18
l1_mm = 1e3*0.8e-6/(2*np.pi)
t1_ps = 1e12*0.8e-6/(2*np.pi)/C.c

def get_freq_time_data(A,plot_ext):
    # Nodes are like [-2,-1,0,1] and walls are like [-2.5,-1.5,-.5,.5,1.5]
    dw = (plot_ext[1] - plot_ext[0])/A.shape[0]
    wc = 0.5*plot_ext[0] + 0.5*(plot_ext[1] + dw)
    tmax = 2*np.pi/dw
    return dw,wc,tmax

A = np.load(base_diagnostic+'_'+searay_object_name+'_'+searay_wave_type+'_wave.npy')
plot_ext = np.load(base_diagnostic+'_'+searay_object_name+'_'+searay_wave_type+'_plot_ext.npy')
plot_ext = np.concatenate((plot_ext[0:2],l1_mm*plot_ext[2:4]))
print(A.shape)
print(plot_ext)
if time_domain and real_field:
    coeff = (plot_ext[1] - plot_ext[0])/(2*np.pi)
    A = coeff*np.fft.ifftshift(A,axes=0)
    A = np.conj(A)
    A = np.fft.ifft(A,axis=0)
    dw,wc,tmax = get_freq_time_data(A,plot_ext)
    plot_ext[0] = 0.0
    plot_ext[1] = t1_ps*tmax
    hstr = r'$t-z/c$ (ps)'
    vstr = r'$\rho$ (mm)'
    cstr = r'$A(t)$'
    t_nodes = np.linspace(0.0,tmax,A.shape[0])[...,np.newaxis,np.newaxis,np.newaxis]
    plot_data = np.real(A*np.exp(-1j*wc*t_nodes))
if time_domain and not real_field:
    coeff = (plot_ext[1] - plot_ext[0])/(2*np.pi)
    A = coeff*np.fft.ifftshift(A,axes=0)
    A = np.conj(A)
    A = np.fft.ifft(A,axis=0)
    dw,wc,tmax = get_freq_time_data(A,plot_ext)
    plot_ext[0] = 0.0
    plot_ext[1] = t1_ps*tmax
    hstr = r'$t-z/c$ (ps)'
    vstr = r'$\rho$ (mm)'
    cstr = r'$|a|(t)$'
    plot_data = np.abs(A)
if not time_domain:
    hstr = r'$\omega/\omega_0$'
    vstr = r'$\rho$ (mm)'
    cstr = r'$|a|(\omega)$'
    plot_data = np.abs(A)

def disp_frame(frame):

    yslice = np.int(A.shape[2]/2)
    plt.figure(1,figsize=(10,7))
    plt.imshow(plot_data[:,:,yslice,frame].swapaxes(0,1),vmin=np.min(plot_data),vmax=np.max(plot_data),origin='lower',cmap='viridis',aspect='auto',extent=plot_ext)
    b=plt.colorbar()
    b.set_label(cstr,size=24)
    plt.xlabel(hstr,size=24)
    plt.ylabel(vstr,size=24)

    plt.tight_layout()
    plt.show()

ipywidgets.interact(disp_frame,frame=ipywidgets.IntSlider(min=0,max=plot_data.shape[3]-1,step=1,value=0,continuous_update=False));

(64, 64, 64, 9)
[ 0.30985113  1.66891352 -0.45836624  0.45836624]


interactive(children=(IntSlider(value=0, continuous_update=False, description='frame', max=8), Output()), _dom…